# Predicción del tráfico

En este proyecto, se cuenta con una base de datos obtenida de la plataforma *Kaggle*, la cual cuenta con datos sobre el tráfico en intervalos de 15 minutos durante las 24 horas del día, los 30 días del mes. Estos datos corresponden a la cantidad de carros, buses, camiones y motos que hay en los distintos tiempos del día anteiormente.

## Objetivo general del proyecto.


*   Crear un modelo de machine learning lo suficientemente capaz de clasificar y predecir el nivel de tráfico que puede llegar a resultar en un lugar teniendo en cuenta el día, la hora y el nivel de vehiculos transitando.

### Objetivos específicos del proyecto.


*   Analizar los datos, limpiarlos y procesarlos
*   Usar las herramientas necesarias para entender los datos
*   Crear un modelo de machine learning que se ajuste bien a los datos








# Entendimiento de los datos

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_csv("Traffic.csv")

In [ ]:
df.head()

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,Traffic Situation
0,12:00:00 AM,10,Tuesday,31,0,4,4,39,low
1,12:15:00 AM,10,Tuesday,49,0,3,3,55,low
2,12:30:00 AM,10,Tuesday,46,0,3,6,55,low
3,12:45:00 AM,10,Tuesday,51,0,2,5,58,low
4,1:00:00 AM,10,Tuesday,57,6,15,16,94,normal


In [ ]:
df.shape

(2976, 9)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2976 entries, 0 to 2975
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Time               2976 non-null   object
 1   Date               2976 non-null   int64 
 2   Day of the week    2976 non-null   object
 3   CarCount           2976 non-null   int64 
 4   BikeCount          2976 non-null   int64 
 5   BusCount           2976 non-null   int64 
 6   TruckCount         2976 non-null   int64 
 7   Total              2976 non-null   int64 
 8   Traffic Situation  2976 non-null   object
dtypes: int64(6), object(3)
memory usage: 209.4+ KB


In [ ]:
df.isnull().any()

Time                 False
Date                 False
Day of the week      False
CarCount             False
BikeCount            False
BusCount             False
TruckCount           False
Total                False
Traffic Situation    False
dtype: bool

Al observar de que forma están compuestos los datos, podemos darnos cuenta en primer lugar que no hay datos faltantes. En segundo lugar podemos darnos cuenta que hay dos variables que son discretas: "Traffic situation" y "Day of the Week". También tenemos un varaible que aunque esta códificada como entero en realidad es de tipo fecha, la cual es "Time".

Debido a la anterior, más adelante debemos transformar los datos a niveles que puedan ser procesados por los modelos.

In [ ]:
df.describe()

,Date,CarCount,BikeCount,BusCount,TruckCount,Total
count,2976.000000,2976.000000,2976.000000,2976.000000,2976.000000,2976.000000
mean,16.000000,68.696573,14.917339,15.279570,15.324933,114.218414
std,8.945775,45.850693,12.847518,14.341986,10.603833,60.190627
min,1.000000,6.000000,0.000000,0.000000,0.000000,21.000000
25%,8.000000,19.000000,5.000000,1.000000,6.000000,55.000000
50%,16.000000,64.000000,12.000000,12.000000,14.000000,109.000000
75%,24.000000,107.000000,22.000000,25.000000,23.000000,164.000000
max,31.000000,180.000000,70.000000,50.000000,40.000000,279.000000


## Entendimiento de los datos a partir de un análisis gráfico

En esta sección se pretende darle una mirada gráfica a los datos, con el objetivo de poder entender la forma en que se comportan los datos y poder extraer posible información relevante que nos ayude en el momento de obtener los resultados de los modelos, a poder contrastar la información obetenida en este punto con la que obtendremos más adelante.

In [ ]:

fig = px.bar(df, x = df["Total"], y = df["Day of the week"], color = "Traffic Situation", title = "Trafico por día")
fig.show()

Aquí pasa algo muy interesante. Podemos darnos cuenta que la situación del tráfico no depende en una forma deterministica de la cantidad de automotores circulando en la vía. Es decir, podemos fijarnos en la cantidad exacta de automotores en la cual el tráfico pasa de estar en un clase a otra (low, normal, heavy, high), la cual es diferente para cada día.

Lo anterior puede deberse a que la diferencia que puede tener un carro, entre un bus o un camión, puede tener incidencia en el tráfico que puede ser generado. Esto es importante tenerlo en cuenta, porque en el momento que deseemos crear, evaluar y predecir nuestro modelo, podemos tener una idea sobre cuales son las posilbes variables que más lo están afectando.

In [ ]:
df.Date.value_counts()

10    96
26    96
8     96
7     96
6     96
5     96
4     96
3     96
2     96
1     96
31    96
30    96
29    96
28    96
27    96
25    96
11    96
24    96
23    96
22    96
21    96
20    96
19    96
18    96
17    96
16    96
15    96
14    96
13    96
12    96
9     96
Name: Date, dtype: int64

In [ ]:
dia_10 = df[df['Date'] == 10]


In [ ]:
dias_unicos = df.Date.unique()
for dia in dias_unicos:
    dia_data = df[df['Date'] == dia]

    dia_semana = dia_data['Day of the week'].iloc[0]  # Obtiene el valor de 'Day of the week' para el día actual

    fig = px.line(dia_data, x='Time', y='Total', title=f'Gráfico para el Día {dia}, {dia_semana}')
    fig.show()

En esta ocasión, podemos darnos cuenta de que el tráfico no se comporta de forma constante a lo largo de los días. Esto no es algo nuevo, ya que todos podemos intuir que cada día no es para nada igual al siguiente. Lo que si es importante, es ver que los días como viernes o sabádos tienen un comportamiento similar, esto puede deberse al comportamiento social ya que días como viernes o sabádos, son días de alta fricción social sobretodo en horas de la tarde hasta la madrugada.



## Prepocesamiento de los datos.

En este apartado, modificaremos todos los datos necesarios de nuestro data frame para posteriormente ser procesados y modelados.

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

df["Day"] = encoder.fit_transform(df["Day of the week"])
df["Traffic_Situation"] = encoder.fit_transform(df["Traffic Situation"])
df.drop(["Day of the week","Traffic Situation"],inplace = True, axis = 1)
df.head()

,Time,Date,CarCount,BikeCount,BusCount,TruckCount,Total,Day,Traffic_Situation
0,12:00:00 AM,10,31,0,4,4,39,5,2
1,12:15:00 AM,10,49,0,3,3,55,5,2
2,12:30:00 AM,10,46,0,3,6,55,5,2
3,12:45:00 AM,10,51,0,2,5,58,5,2
4,1:00:00 AM,10,57,6,15,16,94,5,3


In [ ]:
df['Time'] = pd.to_datetime(df['Time']).dt.hour * 60 + pd.to_datetime(df['Time']).dt.minute
df['Time'] = df['Time'] / 60.0
df.head()


,Time,Date,CarCount,BikeCount,BusCount,TruckCount,Total,Day,Traffic_Situation
0,0.00,10,31,0,4,4,39,5,2
1,0.25,10,49,0,3,3,55,5,2
2,0.50,10,46,0,3,6,55,5,2
3,0.75,10,51,0,2,5,58,5,2
4,1.00,10,57,6,15,16,94,5,3


In [ ]:
df.to_csv('Traffi.csv', index=False)

# Construcción del modelo

## División de los datos

In [ ]:
from sklearn.model_selection import train_test_split
X, y = df.iloc[:,:-1],df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.3,
                                                    random_state=42)
y

0       2
1       2
2       2
3       2
4       3
       ..
2971    3
2972    3
2973    3
2974    3
2975    3
Name: Traffic_Situation, Length: 2976, dtype: int64

## Modelos y selección.

Para hacer la tarea de clasificación se ha seleccionado los siguientes modelos:



*   Logistic Regression
*   SVM classification
*   Random Forest

Para expirementar con los modelos, se ha decidido usar el método de GridSearch en combinación con Cross-Validation.


## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga']
}

grid_search_3 = GridSearchCV(estimator=lr,
                           param_grid=param_grid,
                           cv=5)
grid_search_3.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.10/dist-packages/sklearn/

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag',
                                    'saga']})

In [ ]:
grid_search_3.best_params_

{'C': 0.01, 'solver': 'newton-cg'}

In [ ]:
grid_search_3.best_score_

0.8967833425567239

In [ ]:
prediction_3 = grid_search_3.predict(X_test)


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:

param_grid = {
    'n_estimators': [300, 400, 500],
}


grid_search_2 = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           cv=5)

grid_search_2.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [300, 400, 500]})

In [ ]:
grid_search_2.best_params_

{'n_estimators': 400}

In [ ]:
grid_search_2.best_score_

0.9956799944659658

In [ ]:
prediction_2 = grid_search_2.predict(X_test)


## SVC

In [ ]:
from sklearn.svm import SVC
m_svc = SVC()
m_svc.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
param_grid = {'C': [10,20,30],
              'kernel': ['linear'],
              'gamma': [1000,1500,2000]}

grid_clf = GridSearchCV(m_svc,
                        param_grid=param_grid,
                        verbose=1,
                        return_train_score=True)


grid_clf.fit(X_train, y_train)



Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(estimator=SVC(),
             param_grid={'C': [10, 20, 30], 'gamma': [1000, 1500, 2000],
                         'kernel': ['linear']},
             return_train_score=True, verbose=1)

In [ ]:
grid_clf.best_params_

{'C': 10, 'gamma': 1000, 'kernel': 'linear'}

In [ ]:
grid_clf.best_score_

0.9015829643977126

In [ ]:
prediction = grid_clf.predict(X_test)

## Métricas

Para medir los modelos, hemos usado métricas para modelos multiclase. Esto debido a que nuestra variable objetivo o target tiene 3 clases.

Para medir los modelos usaremos:


*   Recall
*   Precision
*   F1-Score


Aplicando el método de weighted, debido a que se quiere encontrarla precisión por cada clase, teniendo en cuenta el peso de cada una de ellas.


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print(f"Precisión ponderada de LR: {precision_score(y_test, prediction_3, average='weighted'):.4f}")
print(f"Recall ponderada de LR: {recall_score(y_test, prediction_3, average='weighted'):.4f}")
print(f"Precisión ponderada de LR: {precision_score(y_test, prediction_3, average='weighted'):.4f}\n")

print(f"Precisión ponderada de RF: {precision_score(y_test, prediction_3, average='weighted'):.4f}")
print(f"Recall ponderada de RF: {recall_score(y_test, prediction_3, average='weighted'):.4f}")
print(f"Precisión ponderada de RF: {precision_score(y_test, prediction_2, average='weighted'):.4f}\n")

print(f"Precisión ponderada de SVC: {precision_score(y_test, prediction, average='weighted'):.4f}")
print(f"Recall ponderada de SVC: {recall_score(y_test, prediction, average='weighted'):.4f}")
print(f"F1 ponderada de SVC: {f1_score(y_test, prediction, average='weighted'):.4f}")



Precisión ponderada de LR: 0.9036
Recall ponderada de LR: 0.9048
Precisión ponderada de LR: 0.9036

Precisión ponderada de RF: 0.9036
Recall ponderada de RF: 0.9048
Precisión ponderada de RF: 0.9922

Precisión ponderada de SVC: 0.8998
Recall ponderada de SVC: 0.8992
F1 ponderada de SVC: 0.8993


# Análisis de los modelos.

Al momento de verificar las metricas resultante de los modelos, podemos observar que los resultados entre el modelo random forest y logistic regression son identicos. A diferencia de los resultados para el modelo de support vector machine que es minimamente más bajo.

Sabiendo lo anterior, de entrada podemos descartar el modelo maquina de soporte vectorial, ya que sus metricas son menores y más importante que eso, el tiempo en ejecutarse el modelo es bastante considerable respecto a los otros dos modelos.

Al tratar de decidir entre el modelo de logistic regression y el modelo random forest, al ser sus metricas iguales, podriamos determinar la elección del modelo en base al tiempo de ejecución, el cual, en esta ocasión es de 4s menos, que en datos muy grandes puede resultar bastante ventajoso.